In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from collections import defaultdict
import random

from sklearn.feature_extraction.text import TfidfVectorizer
import urllib
import scipy.optimize
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kennnnnnn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
rent = pd.read_csv('clean.csv')
rent.review_summary = rent.review_summary.fillna('')
rent.review_text = rent.review_text.fillna('')

In [3]:
rent = rent.sample(frac=1).reset_index(drop=True)
train = rent[:180000]
valid = rent[180000:]

In [4]:
y_train = train.fit
y_valid = valid.fit

In [5]:
np.unique(rent['rented for'])

array(['date', 'everyday', 'formal affair', 'other', 'party',
       'party: cocktail', 'vacation', 'wedding', 'work'], dtype=object)

In [6]:
np.unique(rent['body type'])

array(['apple', 'athletic', 'full bust', 'hourglass', 'other', 'pear',
       'petite', 'straight & narrow'], dtype=object)

In [7]:
np.unique(rent['bust_char'])

array(['a', 'aa', 'b', 'c', 'd', 'd+', 'dd', 'ddd/e', 'f', 'g', 'h', 'i',
       'j'], dtype=object)

In [8]:
np.unique(rent['category'])

array(['bot', 'long', 'top_long', 'top_short'], dtype=object)

In [9]:
rent

fit  user_id  item_id  weight  rating     rented for  \
0         fit   351011  1076484  4.9885     8.0        wedding   
1       large   313762   921642 -3.1745     8.0        wedding   
2       large   252800   132738 -0.9070    10.0  formal affair   
3         fit   429675   145906  0.0000    10.0        wedding   
4       large   149493  2284114  5.8955     8.0           work   
...       ...      ...      ...     ...     ...            ...   
192539  large   178433  1436642 -5.4420     6.0       vacation   
192540    fit   788097   475151  0.0000     8.0          other   
192541    fit   366010   295362 -5.4420    10.0          other   
192542    fit   434105   985499  5.8955     8.0  formal affair   
192543    fit    96749  2268666 -9.9770    10.0       vacation   

                                              review_text  body type  \
0       A little tight through the waist...still recov...  hourglass   
1       Pretty dress. Runs large in the bust. I wore t...      other   
2       Hey Guys,\n\nI wore this to the 2012, 54th Gra...  hourglass   
3       The dress does run long, even though I was wea...     petite   
4       cute, but ran a little bit big. i think i woul...  hourglass   
...                                                   ...        ...   
192539  This dress is a thicker material and pretty co...     petite   
192540  A fun rental of a floaty feminine dress. There...   athletic   
192541  Fit was perfect.  Very comfortable to wear.  T...       pear   
192542  It fit true to size and I am glad I got the "l...      other   
192543  Adorable for cover up...free flowing and comfo...  full bust   

                                           review_summary   category  height  \
0       Had a great time in the dress. Felt beautiful....       long   -0.79   
1                              Pretty dress, large bust.        long    4.29   
2       I stood out even at such a huge event like the...       long    1.75   
3       Felt amazing in this gorgeous gown at a black ...       long  -13.49   
4                                                Cute top  top_short   -3.33   
...                                                   ...        ...     ...   
192539                                                  .       long   -8.41   
192540              Ethereal, flattering evening dress.         long   -0.79   
192541  I wore this to a charity dinner that was busin...       long   -8.41   
192542  I LOVED this dress and received so many compli...       long    1.75   
192543                                           Vacation       long   -8.41   

             size       age        review_date  bust_num bust_char  
0       -0.245175  3.871201  February 16, 2015        34         b  
1       -4.245175  3.583519      June 27, 2017        34         a  
2      -11.245175  3.367296  February 26, 2012        36         c  
3      -12.245175  3.178054   October 30, 2013        34         c  
4        7.754825  3.218876   October 10, 2017        36         d  
...           ...       ...                ...       ...       ...  
192539  -0.245175  3.688879       July 6, 2017        34         b  
192540  -0.245175  4.043051  November 16, 2017        32         a  
192541   1.754825  3.951244   October 18, 2017        32         d  
192542   0.754825  3.637586    August 10, 2015        36         b  
192543  -8.245175  3.367296     March 22, 2016        34         d  

[192544 rows x 16 columns]

In [10]:
rent[['age']].to_numpy()

array([[3.87120101],
       [3.58351894],
       [3.36729583],
       ...,
       [3.95124372],
       [3.63758616],
       [3.36729583]])

In [11]:
rented_for_one= pd.get_dummies(rent['rented for'], prefix='rented for')
category_one = pd.get_dummies(rent['category'], prefix='category')
bust_char_one = pd.get_dummies(rent['bust_char'], prefix='bust_char')
body_type_one = pd.get_dummies(rent['body type'], prefix='body type')

In [12]:
#rented_for
# rented_for_train = rented_for_one[:180000].to_numpy()
# rented_for_valid = rented_for_one[180000:].to_numpy()

#category
# category_one_train = category_one[:180000].to_numpy()
# category_one_valid = category_one[180000:].to_numpy()

#bust_num
# bust_num_train = train[['bust_num']].to_numpy()
# bust_num_valid = valid[['bust_num']].to_numpy()

#bust_char
# bust_char_for_train = bust_char_one[:180000].to_numpy()
# bust_char_for_valid = bust_char_one[180000:].to_numpy()

#body_type
# body_type_train = body_type_one[:180000].to_numpy()
# body_type_valid = body_type_one[180000:].to_numpy()

#age
# age_train = train[['age']].to_numpy()
# age_valid = valid[['age']].to_numpy()

# size
# size_train = train[['size']].to_numpy()
# size_valid = valid[['size']].to_numpy()

# #weight
# weight_train = train[['weight']].to_numpy()
# weight_valid = valid[['weight']].to_numpy()

# #height
# height_train = train[['height']].to_numpy()
# height_valid = valid[['height']].to_numpy()

# #rating
rating_train = train[['rating']].to_numpy()
rating_valid = valid[['rating']].to_numpy()


In [13]:
tfidf = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_summary = tfidf.fit_transform(train.review_summary).toarray()

In [14]:
tfidf2 = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_text = tfidf2.fit_transform(train.review_text).toarray()

In [15]:
combined_text = np.hstack((Train_X_summary, Train_X_text))

In [16]:
other_feature = rating_train

In [17]:
combined_text.shape

(180000, 2000)

In [18]:
other_feature.shape

(180000, 1)

In [19]:
lstsss = []
for i in range(180000):
    lst = list(combined_text[i]) + list(other_feature[i])
    lstsss.append(lst)

In [20]:
Valid_X_summary = tfidf.transform(valid.review_summary).toarray()

In [21]:
Valid_X_text = tfidf2.transform(valid.review_text).toarray()

In [22]:
combined_text_valid = np.hstack((Valid_X_summary, Valid_X_text))

In [23]:
other_feature_valid = rating_valid

In [24]:
combined_text_valid.shape

(12544, 2000)

In [25]:
other_feature_valid.shape

(12544, 1)

In [26]:
lstsss_valid = []
for i in range(12544):
    lst = list(combined_text_valid[i]) + list(other_feature_valid[i])
    lstsss_valid.append(lst)

In [27]:
final_feature = lstsss

In [28]:
clf = LogisticRegression()
clf.fit(final_feature, y_train)

C:\Users\Kennnnnnn\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
clf.score(lstsss_valid, y_valid)

0.8172831632653061

In [31]:
np.unique(rent['fit'])

array(['fit', 'large', 'small'], dtype=object)

In [32]:
rent_temp = rent

In [39]:
rent_temp['fit'][1]

'large'

In [41]:
for i in rent_temp.index:
    if rent_temp.loc[i,'fit'] == 'small':
        rent_temp.loc[i,'fit'] = 0
    if rent_temp.loc[i,'fit'] == 'fit':
        rent_temp.loc[i,'fit'] = 1
    if rent_temp.loc[i,'fit'] == 'large':
        rent_temp.loc[i,'fit'] = 2

In [47]:
rent_temp[['fit','rating']].corr()

rating
rating     1.0

In [49]:
rent_temp["fit"] = rent_temp["fit"].astype(float)

In [50]:
column_1 = rent_temp["fit"]
column_2 = rent_temp["rating"]
correlation = column_1.corr(column_2)

In [51]:
correlation

0.028358140500538136

In [20]:
y_valid.value_counts()

fit      9235
small    1698
large    1611
Name: fit, dtype: int64

In [21]:
9235 / (len(y_valid))

0.7362085459183674

In [22]:
clf = LogisticRegression(max_iter = 10000, C=0.1)
clf.fit(combined_text, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
clf.score(combined_text_valid, y_valid)

0.8038903061224489